# Chains

> Chains-based functions for PDFs.

In [ ]:
# | default_exp pdf.chains

In [ ]:
# | export

from langchain_ray.imports import *
from langchain_ray.chains import *
from langchain_ray.utils import *
from langchain_ray.pdf.utils import *

In [ ]:
#| hide

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [ ]:
# | export


def pdf_to_docs_chain(
    splitter=None,
    chunk_size=200,
    chunk_overlap=20,
    separators=["\n\n", "\n", "(?<=\. )", " ", ""],
    add_start_index=True,
    proc=True,
    input_variables=["path"],
    output_variables=["docs"],
    verbose=False,
):
    """Chain that returns a list of `Documents` extracted from a PDF path.
    The path can be a single PDF path or a list of paths or a directory path."""
    return transform_chain(
        pdf_to_docs,
        transform_kwargs=dict(
            splitter=splitter,
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            separators=separators,
            add_start_index=add_start_index,
            proc=proc,
        ),
        vars_kwargs_mapping={input_variables[0]: "path"},
        input_variables=input_variables,
        output_variables=output_variables,
        verbose=verbose,
    )


def add_cats_to_docs_chain(
    cats_model,
    input_variables=["docs"],
    output_variables=["cat_docs"],
    verbose=False,
):
    "Chain that adds the categories to a list of `Documents` using `cats_model`."
    return transform_chain(
        add_cats_to_docs,
        transform_kwargs=dict(cats_model=cats_model),
        input_variables=input_variables,
        output_variables=output_variables,
        vars_kwargs_mapping={input_variables[0]: "docs"},
        verbose=verbose,
    )

In [ ]:
# | eval: false

verbose = True
cats_model = SetFitModel.from_pretrained("HamzaFarhan/PDFSegs").to("cuda:0")
cats_folder = "/media/hamza/data2/faiss_data/saved_cats"
if os.path.exists(cats_folder):
    shutil.rmtree(cats_folder)

In [ ]:
# | eval: false

chain1 = pdf_to_docs_chain(
    input_variables=["path"], output_variables=["docs"], verbose=verbose
)
chain2 = add_cats_to_docs_chain(
    cats_model=cats_model,
    input_variables=["docs"],
    output_variables=["cat_docs"],
    verbose=verbose,
)
chain3 = docs_to_json_chain(
    json_folder=cats_folder,
    input_variables=["cat_docs"],
    output_variables=["json_docs"],
    verbose=verbose,
)
chain = SequentialChain(
    chains=[chain1, chain2, chain3],
    input_variables=["path"],
    output_variables=["json_docs"],
    verbose=verbose,
)

In [ ]:
# | eval: false

pdf = "../../resumes_5/"
res = chain(dict(path=pdf))



> Entering new  chain...


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.


> Entering new  chain...

> Finished chain.

> Finished chain.


In [ ]:
# | eval: false

doc = res["json_docs"][0][-1]
print(doc.metadata)
print(doc.page_content)

{'source': '../../resumes_5/0cf20170-8051-41ba-9060-1a82d43f4289.pdf', 'page': 0, 'start_index': 3474, 'category': 'Education'}
bachelor of commerce (b. com) - university of mumbai 2008 - 2011


In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()